In [1]:
from smartsim import Experiment
from smartsim.database import SlurmOrchestrator
from smartsim.settings import SrunSettings
from smartsim import slurm

def launch_cluster_orc(experiment, port, alloc):
    """Just spin up a database cluster, check the status
       and tear it down"""

    db = SlurmOrchestrator(port=port,
                            db_nodes=1,
                            batch=False,
                            alloc=alloc,
                            interface="ib0")
    

    # generate directories for output files
    # pass in objects to make dirs for
    experiment.generate(db, overwrite=True)

    # start the database on interactive allocation
    experiment.start(db, block=True)

    return db

def create_uploader(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="data_uploader.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    uploader = experiment.create_ensemble("uploader", replicas=2, run_settings=srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    uploader.attach_generator_files(to_copy=["./torch/data_uploader.py"])
    experiment.generate(uploader, overwrite=True)
    return uploader


def create_trainer(experiment, alloc):

    srun = SrunSettings(exe="python",
                        exe_args="training_service.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_tasks(1)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./torch/training_service.py")
    experiment.generate(trainer, overwrite=True)
    return trainer


def create_trainer_hvd(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="training_service_hvd.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./torch/training_service_hvd.py")
    experiment.generate(trainer, overwrite=True)
    return trainer

In [2]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

12:48:20 osprey.us.cray.com SmartSim[49352] INFO Allocation successful with Job ID: 526021


In [ ]:
exp = Experiment("launch_streaming", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 8)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer(exp, alloc)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

12:58:39 osprey.us.cray.com SmartSim[49352] INFO Working in previously created experiment
12:59:13 osprey.us.cray.com SmartSim[49352] INFO Working in previously created experiment
12:59:19 osprey.us.cray.com SmartSim[49352] INFO Working in previously created experiment
12:59:28 osprey.us.cray.com SmartSim[49352] INFO uploader_0(526021.9): New
12:59:28 osprey.us.cray.com SmartSim[49352] INFO uploader_1(526021.10): New
12:59:28 osprey.us.cray.com SmartSim[49352] INFO trainer(526021.11): New
12:59:33 osprey.us.cray.com SmartSim[49352] INFO uploader_0(526021.9): Running
12:59:33 osprey.us.cray.com SmartSim[49352] INFO uploader_1(526021.10): Running
12:59:33 osprey.us.cray.com SmartSim[49352] INFO trainer(526021.11): Running
12:59:38 osprey.us.cray.com SmartSim[49352] INFO uploader_0(526021.9): Running
12:59:38 osprey.us.cray.com SmartSim[49352] INFO uploader_1(526021.10): Running
12:59:38 osprey.us.cray.com SmartSim[49352] INFO trainer(526021.11): Running
12:59:43 osprey.us.cray.com SmartS

In [6]:
exp.stop(db, uploader_model, trainer_model)

12:57:52 osprey.us.cray.com SmartSim[49352] INFO Stopping model trainer with job name trainer-CG3C9ROJ3AS6
12:57:53 osprey.us.cray.com SmartSim[49352] INFO Stopping model uploader_0 with job name uploader_0-CG3C9OO7UM5W
12:57:53 osprey.us.cray.com SmartSim[49352] INFO Stopping model uploader_1 with job name uploader_1-CG3C9OO7VPN3
12:57:53 osprey.us.cray.com SmartSim[49352] INFO Stopping model orchestrator_0 with job name orchestrator_0-CG3C99BKV37R


In [ ]:
exp = Experiment("launch_streaming_hvd", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 8)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer_hvd(exp, alloc, 1, 8)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

13:03:31 osprey.us.cray.com SmartSim[65563] INFO Working in previously created experiment
13:04:05 osprey.us.cray.com SmartSim[65563] INFO Working in previously created experiment
13:04:11 osprey.us.cray.com SmartSim[65563] INFO Working in previously created experiment
13:04:20 osprey.us.cray.com SmartSim[65563] INFO uploader_0(513944.9): New
13:04:20 osprey.us.cray.com SmartSim[65563] INFO uploader_1(513944.10): New
13:04:20 osprey.us.cray.com SmartSim[65563] INFO trainer(513944.11): New
13:04:25 osprey.us.cray.com SmartSim[65563] INFO uploader_0(513944.9): Running
13:04:25 osprey.us.cray.com SmartSim[65563] INFO uploader_1(513944.10): Running
13:04:25 osprey.us.cray.com SmartSim[65563] INFO trainer(513944.11): Running
13:04:30 osprey.us.cray.com SmartSim[65563] INFO uploader_0(513944.9): Running
13:04:30 osprey.us.cray.com SmartSim[65563] INFO uploader_1(513944.10): Running
13:04:30 osprey.us.cray.com SmartSim[65563] INFO trainer(513944.11): Running
13:04:35 osprey.us.cray.com SmartS

In [5]:
slurm.release_allocation(alloc)

12:44:10 osprey.us.cray.com SmartSim[100911] INFO Releasing allocation: 526019
12:44:10 osprey.us.cray.com SmartSim[100911] INFO Successfully freed allocation 526019
